### Downloading Data from Data Portal using Login Credential  V2

#### Using Selenium - By RAHUL 

In [23]:
import os, time, csv, glob
from datetime import datetime, timedelta

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException

In [24]:
# ---------------- CONFIG ----------------
CSV_PATH = r"C:\Users\Administrator\LIS\MISAutomation\downloads\creds.csv"        # <-- change to your CSV path (A2: username, B2: password)
DOWNLOAD_DIR = os.path.abspath("./downloads")  # downloaded file target
BASE_URL = "https://appv2.loveinstore.com/Home"
MAX_WAIT_SECS = 45
HEADLESS       = True

# ----------------------------------------

os.makedirs(DOWNLOAD_DIR, exist_ok=True)


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
End-to-end Selenium automation that:
1) Reads username/password from a CSV (A2=username, B2=password)
2) Logs into the site
3) Navigates to Monthly Attendance (Datewise), sets date range (Month start -> yesterday)
4) Triggers "Download Now" -> "Master Export"
5) Waits for a file to download, loads it into a pandas DataFrame, and (optionally) deletes it

Adjust BASE_URL, CSV_PATH, DOWNLOAD_DIR as needed.
"""

# =========================
# Imports
# =========================
import os
import csv
import glob
import time
from datetime import datetime, timedelta

import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    StaleElementReferenceException,
)

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options


# =========================
# Config (edit these or set as env vars)
# =========================
BASE_URL       = os.getenv("BASE_URL", "https://example.com/")  # <-- put the real URL
CSV_PATH = r"C:\Users\Administrator\LIS\MISAutomation\downloads\creds.csv" 
# CSV_PATH       = os.getenv("CSV_PATH", "./creds.csv")
DOWNLOAD_DIR   = os.getenv("DOWNLOAD_DIR", os.path.abspath("./downloads"))
HEADLESS       = os.getenv("HEADLESS", "1") == "1"
MAX_WAIT_SECS  = int(os.getenv("MAX_WAIT_SECS", "60"))

# Return a DataFrame instead of just a file on disk
RETURN_AS_DF    = True
# If True, keep the downloaded file; if False, delete after reading into DataFrame
KEEP_DOWNLOADED = False


# =========================
# Credential reader
# =========================
def read_creds(csv_path, row_index=1, col_user=0, col_pass=1):
    with open(csv_path, 'r', encoding='utf-8-sig', newline='') as f:
        sample = f.read(2048)
        f.seek(0)
        try:
            dialect = csv.Sniffer().sniff(sample)
        except Exception:
            dialect = csv.excel
        reader = csv.reader(f, dialect)
        rows = [[c.strip() for c in r] for r in reader if any(c.strip() for c in r)]

    if len(rows) <= row_index:
        raise ValueError(f"Need at least {row_index+1} non-empty rows; got {len(rows)}. Put username in A2, password in B2.")
    row = rows[row_index]
    if len(row) <= max(col_user, col_pass):
        raise ValueError(f"Row {row_index+1} has only {len(row)} columns: {row}. Expect username in col A, password in col B.")
    username, password = row[col_user], row[col_pass]
    if not username or not password:
        raise ValueError(f"Row {row_index+1} missing username/password values: {row}.")
    return username, password


# =========================
# Setup: Chrome
# =========================
def make_driver(download_dir: str, headless: bool = True) -> webdriver.Chrome:
    os.makedirs(download_dir, exist_ok=True)

    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--ignore-certificate-errors")
    chrome_options.add_argument("--allow-insecure-localhost")
    chrome_options.add_argument("--ignore-ssl-errors=yes")

    # -------------------------
    # DOWNLOAD PREFS
    # Keep this block enabled to control the download directory.
    # If you want to avoid setting a dir explicitly, you can comment it out.
    prefs = {
        "download.default_directory": download_dir,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True,
    }
    chrome_options.add_experimental_option("prefs", prefs)
    # -------------------------

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver


# =========================
# Helper functions
# =========================
def js_click(driver, el):
    driver.execute_script("arguments[0].click();", el)

def safe_click(driver, locator, timeout=40, attempts=3):
    for attempt in range(attempts):
        el = WebDriverWait(driver, timeout).until(EC.element_to_be_clickable(locator))
        try:
            el.click()
            return el
        except Exception:
            try:
                js_click(driver, el)
                return el
            except StaleElementReferenceException:
                if attempt == attempts - 1:
                    raise
                time.sleep(0.3)
    raise TimeoutException(f"Could not click {locator}")

def switch_into_frame_containing(driver, locator, timeout=30):
    driver.switch_to.default_content()
    # already visible in default?
    try:
        WebDriverWait(driver, 3).until(EC.presence_of_element_located(locator))
        return False
    except TimeoutException:
        pass
    # scan iframes
    frames = driver.find_elements(By.TAG_NAME, "iframe")
    for fr in frames:
        try:
            driver.switch_to.default_content()
            driver.switch_to.frame(fr)
            WebDriverWait(driver, 3).until(EC.presence_of_element_located(locator))
            return True
        except Exception:
            continue
    driver.switch_to.default_content()
    return False

def wait_for_url_contains(driver, part, timeout=40):
    WebDriverWait(driver, timeout).until(lambda d: part in d.current_url)

def set_date_input(driver, actions, input_el, value_ddmmyyyy):
    driver.execute_script("arguments[0].removeAttribute('readonly');", input_el)
    input_el.clear()
    input_el.click()
    input_el.send_keys(value_ddmmyyyy)
    actions.move_by_offset(0, 120).click().perform()  # blur

def find_first(driver, locators, timeout_each=20):
    last_err = None
    for loc in locators:
        try:
            return WebDriverWait(driver, timeout_each).until(EC.presence_of_element_located(loc))
        except Exception as e:
            last_err = e
    raise last_err

def wait_for_download(dir_path, timeout=240):
    start = time.time()
    seen = set(glob.glob(os.path.join(dir_path, "*")))
    while time.time() - start < timeout:
        time.sleep(1)
        current = set(glob.glob(os.path.join(dir_path, "*")))
        new_files = list(current - seen)
        if new_files:
            # wait until any .crdownload finishes
            while any(p.endswith(".crdownload") for p in new_files):
                time.sleep(1)
                current = set(glob.glob(os.path.join(dir_path, "*")))
                new_files = list(current - seen)
            # return the most recently modified new file
            return max(new_files, key=os.path.getmtime)
    return None

def export_file_to_df(file_path: str) -> pd.DataFrame:
    """
    Try Excel first, then CSV. If extension is unclear, attempt both.
    """
    lower = file_path.lower()
    try_order = []
    if lower.endswith((".xlsx", ".xls")):
        try_order = ["excel", "csv"]
    elif lower.endswith(".csv"):
        try_order = ["csv", "excel"]
    else:
        try_order = ["excel", "csv"]

    last_err = None
    for mode in try_order:
        try:
            if mode == "excel":
                return pd.read_excel(file_path, engine=None)
            else:
                return pd.read_csv(file_path)
        except Exception as e:
            last_err = e
            continue
    raise RuntimeError(f"Could not parse '{file_path}' as Excel or CSV. Last error: {last_err}")


# =========================
# Main flow
# =========================
def run_export_flow() -> pd.DataFrame:
    username, password = read_creds(CSV_PATH)
    driver = make_driver(DOWNLOAD_DIR, headless=HEADLESS)
    wait    = WebDriverWait(driver, MAX_WAIT_SECS)
    actions = ActionChains(driver)

    df_export = None
    downloaded = None

    try:
        print("Opening site…")
        driver.get(BASE_URL)

        # --- Login ---
        print("Logging in…")
        username_field = wait.until(EC.presence_of_element_located((By.ID, "UserName")))
        password_field = wait.until(EC.presence_of_element_located((By.ID, "Password")))
        login_button   = wait.until(EC.element_to_be_clickable((By.ID, "sign")))
        driver.execute_script("arguments[0].removeAttribute('readonly');", username_field)
        driver.execute_script("arguments[0].removeAttribute('readonly');", password_field)
        username_field.clear(); username_field.send_keys(username)
        password_field.clear(); password_field.send_keys(password)
        login_button.click()

        # Wait for a post-login signal
        wait.until(EC.any_of(
            EC.presence_of_element_located((By.XPATH, "//aside|//div[contains(@class,'sidebar')]")),
            EC.presence_of_element_located((By.XPATH, "//a[contains(.,'Logout') or contains(@href,'Logout')]"))
        ))
        time.sleep(1.0)

        # --- Data Management ---
        print("Opening Data Management…")
        safe_click(driver, (
            By.XPATH,
            "//a[contains(@href,'/Admin/DataManagement/Index') and .//span[contains(text(),'Data Management')]]"
        ), timeout=40)

        # --- Attendance ---
        print("Opening Attendance…")
        attendance_locators = [
            (By.XPATH, "//a[contains(.,'Attendance') and not(contains(@href,'MonthlyAttendance'))]"),
            (By.XPATH, "//a[contains(@href,'/Attendance') and contains(.,'Attendance')]"),
        ]
        clicked = False
        for loc in attendance_locators:
            try:
                safe_click(driver, loc, timeout=40)
                clicked = True
                break
            except Exception:
                continue
        if not clicked:
            raise TimeoutException("Could not open Attendance (all locators failed).")

        # --- Monthly Attendance ---
        print("Opening Monthly Attendance…")
        monthly_locators = [
            (By.XPATH, "//a[contains(@href,'/AttendanceV2/MonthlyAttendanceV2') and contains(.,'Monthly Attendance')]"),
            (By.XPATH, "//a[contains(.,'Monthly Attendance')]"),
        ]
        clicked = False
        for loc in monthly_locators:
            try:
                el = WebDriverWait(driver, 40).until(EC.presence_of_element_located(loc))
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
                try:
                    el.click()
                except Exception:
                    js_click(driver, el)
                clicked = True
                break
            except Exception:
                continue
        if not clicked:
            raise TimeoutException("Could not open Monthly Attendance (all locators failed).")

        # Confirm navigation or probe DOM
        try:
            wait_for_url_contains(driver, "MonthlyAttendance", timeout=40)
        except Exception:
            pass

        # --- Page ready (iframe-aware) ---
        target_dropdown = (By.XPATH, "//*[@id='Sdatewise' or @name='Sdatewise' or @id='select2-Sdatewise-container']")
        switch_into_frame_containing(driver, target_dropdown, timeout=30)
        WebDriverWait(driver, 40).until(EC.presence_of_element_located(target_dropdown))

        # --- Select 'Datewise' (Select2 robust) ---
        print("Selecting Datewise…")
        try:
            select2_container = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.XPATH, "//span[@id='select2-Sdatewise-container']"))
            )
            select2_container.click()
            # use select2 search + Enter
            search_box = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//input[contains(@class,'select2-search__field')]"))
            )
            search_box.clear()
            search_box.send_keys("Datewise")
            time.sleep(0.3)
            search_box.send_keys(Keys.ENTER)
        except TimeoutException:
            # fallback: direct LI
            try:
                datewise_option = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//li[contains(@id,'select2-Sdatewise') and normalize-space()='Datewise']"))
                )
                js_click(driver, datewise_option)
            except Exception as e:
                print("Select2 fallback failed:", e)

        # --- Dates ---
        first_day = datetime.today().replace(day=1).strftime("%d/%m/%Y")
        yesterday = (datetime.today() - timedelta(days=1)).strftime("%d/%m/%Y")
        print(f"Setting dates: {first_day} → {yesterday}")

        from_input = find_first(driver, [
            (By.ID, "Sdate"),
            (By.NAME, "DateFrom"),
            (By.XPATH, "//input[@placeholder='From' or @placeholder='Date From']")
        ])
        to_input = find_first(driver, [
            (By.ID, "dateto"),
            (By.NAME, "DateTo"),
            (By.XPATH, "//input[@placeholder='To' or @placeholder='Date To']")
        ])

        set_date_input(driver, actions, from_input, first_day)
        set_date_input(driver, actions, to_input, yesterday)
        time.sleep(0.5)

        # --- Download Now -> Master Export ---
        print("Triggering download…")
        download_dropdown_locators = [
            (By.XPATH, "//a[contains(@class,'dropdown-toggle')][contains(.,'Download Now')]"),
            (By.XPATH, "//button[contains(@class,'dropdown-toggle')][contains(.,'Download Now')]"),
        ]
        for loc in download_dropdown_locators:
            try:
                safe_click(driver, loc)
                break
            except Exception:
                continue
        else:
            raise TimeoutException("Could not open 'Download Now' dropdown.")

        master_export_locators = [
            (By.XPATH, "//button[@type='submit' and contains(@class,'dropdown-item') and @value='Export' and contains(.,'Master Export')]"),
            (By.XPATH, "//button[contains(.,'Master Export') and (@type='submit' or @value='Export')]"),
            (By.XPATH, "//a[contains(.,'Master Export')]"),
        ]
        for loc in master_export_locators:
            try:
                safe_click(driver, loc)
                break
            except Exception:
                continue
        else:
            driver.save_screenshot("export_button_not_found.png")
            raise TimeoutException("Could not click Master Export.")

        # --- Wait for file ---
        downloaded = wait_for_download(DOWNLOAD_DIR, timeout=240)
        if not downloaded:
            raise TimeoutError("Timed out waiting for the export file to download.")
        print(f"✅ Download complete: {downloaded}")

        # --- Read into DataFrame and optionally clean up ---
        if RETURN_AS_DF:
            df_export = export_file_to_df(downloaded)
            print(f"📄 DataFrame loaded. Shape: {df_export.shape}")
            if not KEEP_DOWNLOADED:
                try:
                    os.remove(downloaded)
                    print("🧹 Downloaded file removed after reading into DataFrame.")
                except Exception:
                    pass

    finally:
        try:
            driver.quit()
        except Exception:
            pass

    # If RETURN_AS_DF=False you can return the filepath instead
    if RETURN_AS_DF:
        if df_export is None:
            raise RuntimeError("Expected a DataFrame, but df_export is None.")
        return df_export
    else:
        return downloaded


# =========================
# Entrypoint
# =========================
if __name__ == "__main__":
    df = run_export_flow()
    if isinstance(df, pd.DataFrame):
        print("Final DF shape:", df.shape)
        # Example: show a few columns/rows
        with pd.option_context("display.max_columns", None, "display.width", 200):
            print(df.head(5))
    else:
        print("File saved at:", df)


Opening site…
Logging in…
